In [ ]:
# data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, TimeDistributed, Conv2D, MaxPooling2D, Flatten, SimpleRNN, LSTM, Dropout, Dense, BatchNormalization
from sklearn.model_selection import train_test_split

In [ ]:
# Path to the folder containing the good and bad exercise videos
video_folder = '/content/drive/My Drive/APS360H1/data'  # Adjust this path as necessary

img_height, img_width = 64, 64  # Dimensions for resizing frames
sequence_length = 30  # Number of frames to consider in each sequence

In [ ]:
def extract_frames_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_height, img_width))
        frame = img_to_array(frame)
        frames.append(frame)
        frame_count += 1
        if frame_count == sequence_length:
            break
    cap.release()
    return np.array(frames)

def load_videos(video_folder):
    X, y_class, y_assessment = [], [], []
    assessment_classes = os.listdir(video_folder)

    for assessment in assessment_classes:
        assessment_folder = os.path.join(video_folder, assessment)
        if not os.path.isdir(assessment_folder):  # Skip non-directory files
            continue
        class_names = os.listdir(assessment_folder)
        class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}

        for class_name in class_names:
            class_folder = os.path.join(assessment_folder, class_name)
            for video_name in os.listdir(class_folder):
                video_path = os.path.join(class_folder, video_name)
                if not video_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):  # Skip non-video files
                    continue
                frames = extract_frames_from_video(video_path)
                if len(frames) == sequence_length:
                    X.append(frames)
                    y_class.append(class_indices[class_name])
                    y_assessment.append(1 if assessment == 'good' else 0)  # 1 for good, 0 for bad

    X = np.array(X)
    y_class = np.array(y_class)
    y_assessment = np.array(y_assessment)
    if len(X) == 0 or len(y_class) == 0:
        return None, None, None  # Return None if no valid data is found

    y_class = to_categorical(y_class, num_classes=len(class_names))
    y_assessment = np.array(y_assessment)
    return X, y_class, y_assessment

def CNN_LSTM(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # TimeDistributed CNN Layers
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_layer)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)

    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)

    x = TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)

    x = TimeDistributed(Flatten())(x)

    # LSTM Layer
    x = LSTM(128, return_sequences=False)(x)
    x = Dropout(0.5)(x)

    # Dense Layers for Exercise Classification
    classification_output = Dense(128, activation='relu')(x)
    classification_output = Dropout(0.5)(classification_output)
    classification_output = Dense(num_classes, activation='softmax', name='classification_output')(classification_output)

    # Dense Layers for Good/Bad Assessment
    assessment_output = Dense(128, activation='relu')(x)
    assessment_output = Dropout(0.5)(assessment_output)
    assessment_output = Dense(1, activation='sigmoid', name='assessment_output')(assessment_output)

    # Model with two outputs
    model = Model(inputs=input_layer, outputs=[classification_output, assessment_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'categorical_crossentropy', 'assessment_output': 'binary_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'assessment_output': 'accuracy'})

    return model

In [ ]:
def CNN_RNN_LSTM(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # TimeDistributed CNN Layers
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_layer)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)

    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)

    x = TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)

    x = TimeDistributed(Flatten())(x)

    # RNN Layer
    x = SimpleRNN(64, return_sequences=True)(x)
    x = Dropout(0.5)(x)

    # LSTM Layer
    x = LSTM(128, return_sequences=False)(x)
    x = Dropout(0.5)(x)

    # Dense Layers for Exercise Classification
    classification_output = Dense(128, activation='relu')(x)
    classification_output = Dropout(0.5)(classification_output)
    classification_output = Dense(num_classes, activation='softmax', name='classification_output')(classification_output)

    # Dense Layers for Good/Bad Assessment
    assessment_output = Dense(128, activation='relu')(x)
    assessment_output = Dropout(0.5)(assessment_output)
    assessment_output = Dense(1, activation='sigmoid', name='assessment_output')(assessment_output)

    # Model with two outputs
    model = Model(inputs=input_layer, outputs=[classification_output, assessment_output])
    model.compile(optimizer='adam',
                  loss={'classification_output': 'categorical_crossentropy', 'assessment_output': 'binary_crossentropy'},
                  metrics={'classification_output': 'accuracy', 'assessment_output': 'accuracy'})

    return model

In [ ]:
# Load videos
X, y_class, y_assessment = load_videos(video_folder)

In [ ]:
 # Check if data was loaded correctly
if X is None or y_class is None or y_assessment is None:
    raise ValueError("No valid video data found. Please check your video paths and formats.")

# Split the data into training, validation, and test sets
X_train_val, X_test, y_class_train_val, y_class_test, y_assessment_train_val, y_assessment_test = train_test_split(
    X, y_class, y_assessment, test_size=0.15, random_state=42)

X_train, X_val, y_class_train, y_class_val, y_assessment_train, y_assessment_val = train_test_split(
    X_train_val, y_class_train_val, y_assessment_train_val, test_size=0.1765, random_state=42)  # 0.1765 * 0.85 ≈ 0.15

# Build and compile the model
input_shape = (sequence_length, img_height, img_width, 3)
num_classes = len(os.listdir(os.path.join(video_folder, 'good')))
model = CNN_RNN_LSTM(input_shape, num_classes)

# Train the model
history = model.fit(X_train, {'classification_output': y_class_train, 'assessment_output': y_assessment_train},
                    epochs=20, batch_size=8, validation_data=(X_val, {'classification_output': y_class_val, 'assessment_output': y_assessment_val}))

# Evaluate the model
test_loss, test_acc_class, test_acc_assessment = model.evaluate(X_test, {'classification_output': y_class_test, 'assessment_output': y_assessment_test})
print(f'Test Loss: {test_loss}')
print(f'Test Classification Accuracy: {test_acc_class}')
print(f'Test Assessment Accuracy: {test_acc_assessment}')

# Save the model
model.save('/content/drive/My Drive/APS360H1/exercise_assessment_model.h5')

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 737s 13s/step - assessment_output_accuracy: 0.5289 - classification_output_accuracy: 0.2768 - loss: 2.2782 - val_assessment_output_accuracy: 0.6277 - val_classification_output_accuracy: 0.3404 - val_loss: 2.0204
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 745s 13s/step - assessment_output_accuracy: 0.6112 - classification_output_accuracy: 0.3693 - loss: 2.0790 - val_assessment_output_accuracy: 0.5745 - val_classification_output_accuracy: 0.3617 - val_loss: 2.0516
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 703s 12s/step - assessment_output_accuracy: 0.6083 - classification_output_accuracy: 0.3586 - loss: 2.1005 - val_assessment_output_accuracy: 0.5957 - val_classification_output_accuracy: 0.2872 - val_loss: 2.1006
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 685s 12s/step - assessment_output_accuracy: 0.6218 - classification_output_accuracy: 0.3403 - loss: 2.1098 - val_assessment_output_accuracy: 0.5638 - val_classification_output_accuracy: 0.2872 - val_loss: 2.1786


Test Loss: 1.8955925703048706
Test Classification Accuracy: 0.7021276354789734
Test Assessment Accuracy: 0.4893617033958435
